See [guide](https://towardsdatascience.com/audio-deep-learning-made-simple-sound-classification-step-by-step-cebc936bbe5)

In [11]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import pandas as pd
import torch
import wandb
import src.Baseline

from torch.utils.data import DataLoader
from torch.utils.data import random_split
from importlib import reload
from src.Baseline import SoundDS, AudioClassifier, training, inference

In [13]:
torch.cuda.is_available()
if not torch.cuda.is_available():
    raise Exception("GPU not available")

In [14]:
# get all folder names in directory



data_path = "./input/scrape/"

folders = os.listdir(data_path)
folders = [f for f in folders if os.path.isdir(os.path.join(data_path, f))]


current_species_id = 1
df_dict = {"species_id": [], "name": [], "path": []}

for folder in folders:
    files = os.listdir(os.path.join
                       (data_path, folder))
    files = [f for f in files if f.endswith('.mp3')]
    
    if len(files) == 0:
        print(f"No files found in {folder}")
    
    for file in files:
        path = str(os.path.join(data_path, folder, file))
        df_dict["species_id"].append(current_species_id)
        df_dict["name"].append(folder)
        df_dict["path"].append(path)
        
    current_species_id += 1
        
df = pd.DataFrame(df_dict)

df.head()

No files found in Common_Reed_Warbler
No files found in Eurasian_Chaffinch
No files found in Eurasian_Goshawk
No files found in Long-billed_Murrelet
No files found in Long-tailed_Shrike
No files found in Macqueen's_Bustard
No files found in MoltoniΓÇÖs_Warbler
No files found in Rock_SparrowWhite-winged_Snowfinch
No files found in Western_House_Martin


,species_id,name,path
0,1,African_Sacred_Ibis,./input/scrape/African_Sacred_Ibis\XC132389-19...
1,1,African_Sacred_Ibis,./input/scrape/African_Sacred_Ibis\XC132389-19...
2,1,African_Sacred_Ibis,./input/scrape/African_Sacred_Ibis\XC132389-19...
3,1,African_Sacred_Ibis,./input/scrape/African_Sacred_Ibis\XC278223-Sa...
4,1,African_Sacred_Ibis,./input/scrape/African_Sacred_Ibis\XC309388-sa...


In [15]:
df.name.value_counts()

name
Marsh_Warbler            4769
Boreal_Owl               4741
Blyth's_Reed_Warbler     4577
Song_Thrush              4406
Thrush_Nightingale       3968
                         ... 
Bewick's_Swan               2
Ivory_Gull                  2
Houbara_Bustard             1
Pink-backed_Pelican         1
Slender-billed_Curlew       1
Name: count, Length: 428, dtype: int64

In [16]:
# max species id
df.species_id.max()

437

In [17]:
def train():
    wandb.init()

    myds = SoundDS(df, "")

    # Random split of 80:20 between training and validation
    num_items = len(myds)
    num_train = round(num_items * 0.8)
    num_val = num_items - num_train
    train_ds, val_ds = random_split(myds, [num_train, num_val])

    # Create training and validation data loaders
    train_dl = torch.utils.data.DataLoader(train_ds,
                                           batch_size=wandb.config.batch_size_train,
                                           shuffle=True)
    val_dl = torch.utils.data.DataLoader(val_ds,
                                         batch_size=wandb.config.batch_size_val,
                                         shuffle=False)

    # Create the model and put it on the GPU if available
    model = AudioClassifier()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device {device}")
    model = model.to(device)
    # Check that it is on Cuda
    # var = next(model.parameters()).device

    num_epochs = wandb.config.epochs
    training(model, train_dl, num_epochs, device)

    # Run inference on trained model with the validation set
    inference(model, val_dl, device)

    # Save the model in the exchangeable ONNX format
    torch.onnx.export(model, torch.randn(1, 1, 128, 201), "model.onnx")

    wandb.save("model.onnx")

In [ ]:
sweep_config = {
    "name": "Baseline Sweep",
    "method": "bayes",
    "metric": {"goal": "maximize", "name": "val_acc"},
    "parameters": {
        "epochs": {"min": 7, "max": 20},
        "learning_rate": {"min": 0, "max": 0.1, "distribution": "log_uniform"},

        "batch_size_train": {"values": [32]},
        "batch_size_val": {"values": [32]},
        "anneal_strategy": {"values": ["linear"]},
    }
}
sweep_id = wandb.sweep(sweep=sweep_config, project="Baseline-Full", entity="swiss-birder")
wandb.agent(sweep_id, function=train)

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. learning_rate uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: jn8bch69
Sweep URL: https://wandb.ai/swiss-birder/Baseline-Full/sweeps/jn8bch69


wandb: Agent Starting Run: moursmyk with config:
wandb: 	anneal_strategy: linear
wandb: 	batch_size_train: 32
wandb: 	batch_size_val: 32
wandb: 	epochs: 18
wandb: 	learning_rate: 1.0292679668013025


Using device cuda:0
